In [1]:
!pip install wwo-hist
!pip install WorldWeatherPy
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [125]:
from wwo_hist import retrieve_hist_data
from WorldWeatherPy import DetermineListOfAttributes
from WorldWeatherPy import HistoricalLocationWeather
from WorldWeatherPy import RetrieveByAttribute
#Importing Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from joblib import dump, load


### Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/SCC/Data')

In [33]:
allAttributes = DetermineListOfAttributes('bd35020cdd3643f4b69142436222912', verbose = True).retrieve_list_of_options()

Retrieving attribute list...
List of available weather attributes: ['sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination', 'time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherIconUrl', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex']


In [34]:
attributes = ['date','time','tempC', 'weatherDesc','precipMM', 'humidity', 'pressure','cloudcover', 'windspeedKmph', 'visibility', 'uvIndex','moon_illumination']
conditions = ['Sunny','Clear','Cloudy','Rain','Snow']
location_list = ['milan','turin','florence','rome','naples','lecce','palermo']

frequency = 6
start_date = '2018-1-1'
end_date = '2022-12-31'
api_key = 'bd35020cdd3643f4b69142436222912'
csv_directory = ('/content/drive/MyDrive/SCC/Data')

Weather data in Milan 

In [62]:
location = location_list[0]

dataset = RetrieveByAttribute(api_key, attributes, location, start_date, end_date, frequency).retrieve_hist_data()
dataset.to_csv('milan.csv', encoding='utf-8', index=False)  



Retrieving weather data for milan


Retrieving data for milan from: 2018-01-01 to: 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.312641
Retrieving data for milan from: 2018-02-01 to: 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:02.292599
Retrieving data for milan from: 2018-03-01 to: 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:03.341520
Retrieving data for milan from: 2018-04-01 to: 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.387118
Retrieving data for milan from: 2018-05-01 to: 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.492653
Retrieving data for milan from: 2018-06-01 to: 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.544103
Retrieving data for milan from: 2018-07-01 to: 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.611796
Retrieving data for milan from: 2018-08-01 to: 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.678277
Retrieving data for milan from: 2018-09-01 to: 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.722834
Retrieving data for milan from: 2018-10-01 to: 2018-10-31
Time

In [74]:
milanData = pd.read_csv('milan.csv')

milanData = milanData.drop("Unnamed: 0", axis=1)

print(milanData.shape)
milanData.head()

(6669, 13)


,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,5,Rain,0.8,86,1017,100,1,9,1,97,milan
1,2018-01-01,6,4,Rain,3.8,94,1013,100,3,7,1,97,milan
2,2018-01-01,12,5,Rain,2.5,90,1012,58,11,8,1,97,milan
3,2018-01-02,0,4,Cloudy,0.0,81,1010,50,5,10,3,100,milan
4,2018-01-02,6,4,Sunny,0.0,79,1015,0,8,10,3,100,milan


In [75]:
#milanData = pd.read_csv('milan.csv')
tmp = []

for i, row in milanData.iterrows():
  for j in conditions:
    if j.lower() in row['weatherDesc'].lower():
      milanData.at[i, 'weatherDesc'] = j
      break
  else:
      tmp.append(i) if i not in tmp else None
milanData = milanData.drop(tmp)
milanData = milanData.drop_duplicates()
milanData = milanData.reset_index(drop=True)
milanData.to_csv('milan.csv', index=True)

print(milanData.shape)
milanData.head()

(6669, 13)


,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,5,Rain,0.8,86,1017,100,1,9,1,97,milan
1,2018-01-01,6,4,Rain,3.8,94,1013,100,3,7,1,97,milan
2,2018-01-01,12,5,Rain,2.5,90,1012,58,11,8,1,97,milan
3,2018-01-02,0,4,Cloudy,0.0,81,1010,50,5,10,3,100,milan
4,2018-01-02,6,4,Sunny,0.0,79,1015,0,8,10,3,100,milan


In [ ]:
#DA NON ESEGUIRE, PERO' PUO' ESSERE UTILE

milanData = pd.get_dummies(milanData, columns=["weatherDesc"])

from sklearn.preprocessing import StandardScaler

# selezione delle colonne con le caratteristiche da normalizzare
X = milanData[["weatherDesc_Clear", "weatherDesc_Cloudy", "weatherDesc_Rain", "weatherDesc_Sunny"]]
# creazione di un oggetto StandardScaler
scaler = StandardScaler()
# calcolo delle statistiche di standardizzazione (media e deviazione standard) utilizzando i dati di X
scaler.fit(X)
# trasformazione dei dati utilizzando le statistiche appena calcolate
X_scaled = scaler.transform(X)
# conversione del risultato in un DataFrame di Pandas
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled

Weather data in Florence Turin

In [38]:
location = location_list[1]

dataset = RetrieveByAttribute(api_key, attributes, location, start_date, end_date, frequency).retrieve_hist_data()
dataset.to_csv('turin.csv', encoding='utf-8', index=False)  



Retrieving weather data for turin


Retrieving data for turin from: 2018-01-01 to: 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:00.892777
Retrieving data for turin from: 2018-02-01 to: 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:01.724839
Retrieving data for turin from: 2018-03-01 to: 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.626906
Retrieving data for turin from: 2018-04-01 to: 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.519929
Retrieving data for turin from: 2018-05-01 to: 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.366859
Retrieving data for turin from: 2018-06-01 to: 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.267021
Retrieving data for turin from: 2018-07-01 to: 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.142055
Retrieving data for turin from: 2018-08-01 to: 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:07.024996
Retrieving data for turin from: 2018-09-01 to: 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:08.022755
Retrieving data for turin from: 2018-10-01 to: 2018-10-31
Time

In [39]:
turinData = pd.read_csv('turin.csv')
print(turinData.shape)
turinData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,0,[{'value': 'Patchy rain possible'}],0.2,90,1016,33,3,9,2,97,turin
1,2018-01-01,6,4,[{'value': 'Light drizzle'}],0.3,89,1015,92,4,2,2,97,turin
2,2018-01-01,12,8,[{'value': 'Sunny'}],0.1,64,1015,0,4,10,2,97,turin
3,2018-01-01,18,3,[{'value': 'Cloudy'}],0.1,64,1011,87,5,10,2,97,turin
4,2018-01-02,0,2,[{'value': 'Clear'}],0.1,74,1012,3,7,10,3,100,turin


In [40]:
#turinData = pd.read_csv('turin.csv')
tmp = []
conditions = ['Sunny','Clear','Cloudy','Rain','Snow']
for i, row in turinData.iterrows():
  for j in conditions:
    if j.lower() in row['weatherDesc'].lower():
      turinData.at[i, 'weatherDesc'] = j
      break
  else:
      tmp.append(i) if i not in tmp else None
turinData = turinData.drop(tmp)
turinData = turinData.drop_duplicates()
turinData = turinData.reset_index(drop=True)
turinData.to_csv('turin.csv', index=True)

print(turinData.shape)
turinData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,0,Rain,0.2,90,1016,33,3,9,2,97,turin
1,2018-01-01,12,8,Sunny,0.1,64,1015,0,4,10,2,97,turin
2,2018-01-01,18,3,Cloudy,0.1,64,1011,87,5,10,2,97,turin
3,2018-01-02,0,2,Clear,0.1,74,1012,3,7,10,3,100,turin
4,2018-01-02,6,4,Sunny,0.0,56,1017,6,15,10,3,100,turin


Weather data in Florence 

In [41]:
location = location_list[2]

dataset = RetrieveByAttribute(api_key, attributes, location, start_date, end_date, frequency).retrieve_hist_data()
dataset.to_csv('florence.csv', encoding='utf-8', index=False)



Retrieving weather data for florence


Retrieving data for florence from: 2018-01-01 to: 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.111343
Retrieving data for florence from: 2018-02-01 to: 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:01.984724
Retrieving data for florence from: 2018-03-01 to: 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.892317
Retrieving data for florence from: 2018-04-01 to: 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.734809
Retrieving data for florence from: 2018-05-01 to: 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.598939
Retrieving data for florence from: 2018-06-01 to: 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.482671
Retrieving data for florence from: 2018-07-01 to: 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.364650
Retrieving data for florence from: 2018-08-01 to: 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:07.300634
Retrieving data for florence from: 2018-09-01 to: 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:08.123289
Retrieving data for florence fro

In [42]:
florenceData = pd.read_csv('florence.csv')
print(florenceData.shape)
florenceData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,2,[{'value': 'Mist'}],0.0,97,1017,65,6,2,2,97,florence
1,2018-01-01,6,4,[{'value': 'Light drizzle'}],0.7,97,1012,100,8,2,2,97,florence
2,2018-01-01,12,11,[{'value': 'Moderate rain at times'}],2.7,84,1008,63,10,8,2,97,florence
3,2018-01-01,18,5,[{'value': 'Clear'}],0.4,92,1010,11,7,10,2,97,florence
4,2018-01-02,0,4,[{'value': 'Light rain shower'}],0.4,90,1007,51,13,10,2,100,florence


In [43]:
#florenceData = pd.read_csv('florence.csv')
tmp = []
conditions = ['Sunny','Clear','Cloudy','Rain','Snow']
for i, row in florenceData.iterrows():
  for j in conditions:
    if j.lower() in row['weatherDesc'].lower():
      florenceData.at[i, 'weatherDesc'] = j
      break
  else:
      tmp.append(i) if i not in tmp else None
florenceData = florenceData.drop(tmp)
florenceData = florenceData.drop_duplicates()
florenceData = florenceData.reset_index(drop=True)
florenceData.to_csv('florence.csv', index=True)

print(florenceData.shape)
florenceData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,12,11,Rain,2.7,84,1008,63,10,8,2,97,florence
1,2018-01-01,18,5,Clear,0.4,92,1010,11,7,10,2,97,florence
2,2018-01-02,0,4,Rain,0.4,90,1007,51,13,10,2,100,florence
3,2018-01-02,6,5,Rain,1.2,91,1009,49,7,9,2,100,florence
4,2018-01-02,12,13,Sunny,0.0,47,1014,1,9,10,2,100,florence


Weather data in Rome

In [44]:
location = location_list[3]

dataset = RetrieveByAttribute(api_key, attributes, location, start_date, end_date, frequency).retrieve_hist_data()
dataset.to_csv('rome.csv', encoding='utf-8', index=False)



Retrieving weather data for rome


Retrieving data for rome from: 2018-01-01 to: 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.071972
Retrieving data for rome from: 2018-02-01 to: 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:01.935997
Retrieving data for rome from: 2018-03-01 to: 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.828199
Retrieving data for rome from: 2018-04-01 to: 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.733616
Retrieving data for rome from: 2018-05-01 to: 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.632770
Retrieving data for rome from: 2018-06-01 to: 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.614836
Retrieving data for rome from: 2018-07-01 to: 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.439870
Retrieving data for rome from: 2018-08-01 to: 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:07.353880
Retrieving data for rome from: 2018-09-01 to: 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:08.249357
Retrieving data for rome from: 2018-10-01 to: 2018-10-31
Time elapsed (h

In [45]:
romeData = pd.read_csv('rome.csv')
print(romeData.shape)
romeData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,12,[{'value': 'Patchy rain possible'}],0.2,89,1019,45,12,9,3,97,rome
1,2018-01-01,6,11,[{'value': 'Cloudy'}],0.0,87,1016,68,20,10,3,97,rome
2,2018-01-01,12,14,[{'value': 'Patchy rain possible'}],0.9,66,1012,10,24,9,3,97,rome
3,2018-01-01,18,11,[{'value': 'Patchy rain possible'}],1.6,76,1013,4,9,9,3,97,rome
4,2018-01-02,0,10,[{'value': 'Partly cloudy'}],0.0,77,1012,45,15,10,4,100,rome


In [46]:
#romeData = pd.read_csv('rome.csv')
tmp = []
conditions = ['Sunny','Clear','Cloudy','Rain','Snow']
for i, row in romeData.iterrows():
  for j in conditions:
    if j.lower() in row['weatherDesc'].lower():
      romeData.at[i, 'weatherDesc'] = j
      break
  else:
      tmp.append(i) if i not in tmp else None
romeData = romeData.drop(tmp)
romeData = romeData.drop_duplicates()
romeData = romeData.reset_index(drop=True)
romeData.to_csv('rome.csv', index=True)

print(romeData.shape)
romeData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,12,Rain,0.2,89,1019,45,12,9,3,97,rome
1,2018-01-01,6,11,Cloudy,0.0,87,1016,68,20,10,3,97,rome
2,2018-01-01,12,14,Rain,0.9,66,1012,10,24,9,3,97,rome
3,2018-01-01,18,11,Rain,1.6,76,1013,4,9,9,3,97,rome
4,2018-01-02,0,10,Cloudy,0.0,77,1012,45,15,10,4,100,rome


Weather data in Naples

In [47]:
location = location_list[4]

dataset = RetrieveByAttribute(api_key, attributes, location, start_date, end_date, frequency).retrieve_hist_data()
dataset.to_csv('naples.csv', encoding='utf-8', index=False)



Retrieving weather data for naples


Retrieving data for naples from: 2018-01-01 to: 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.038297
Retrieving data for naples from: 2018-02-01 to: 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:02.015951
Retrieving data for naples from: 2018-03-01 to: 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:03.065070
Retrieving data for naples from: 2018-04-01 to: 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.100003
Retrieving data for naples from: 2018-05-01 to: 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.088628
Retrieving data for naples from: 2018-06-01 to: 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.072979
Retrieving data for naples from: 2018-07-01 to: 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.303005
Retrieving data for naples from: 2018-08-01 to: 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.442046
Retrieving data for naples from: 2018-09-01 to: 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.402872
Retrieving data for naples from: 2018-10-01 to: 2018

In [48]:
naplesData = pd.read_csv('naples.csv')
print(naplesData.shape)
naplesData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,13,[{'value': 'Cloudy'}],0.0,85,1021,79,4,10,3,97,naples
1,2018-01-01,6,13,[{'value': 'Partly cloudy'}],0.0,85,1018,44,15,10,3,97,naples
2,2018-01-01,12,14,[{'value': 'Partly cloudy'}],0.0,82,1014,54,25,10,3,97,naples
3,2018-01-01,18,12,[{'value': 'Patchy rain possible'}],0.7,63,1014,35,27,9,3,97,naples
4,2018-01-02,0,11,[{'value': 'Patchy rain possible'}],0.4,72,1014,35,26,9,3,100,naples


In [49]:
#naplesData = pd.read_csv('naples.csv')
tmp = []
conditions = ['Sunny','Clear','Cloudy','Rain','Snow']
for i, row in naplesData.iterrows():
  for j in conditions:
    if j.lower() in row['weatherDesc'].lower():
      naplesData.at[i, 'weatherDesc'] = j
      break
  else:
      tmp.append(i) if i not in tmp else None
naplesData = naplesData.drop(tmp)
naplesData = naplesData.drop_duplicates()
naplesData = naplesData.reset_index(drop=True)
naplesData.to_csv('naples.csv', index=True)

print(naplesData.shape)
naplesData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,13,Cloudy,0.0,85,1021,79,4,10,3,97,naples
1,2018-01-01,6,13,Cloudy,0.0,85,1018,44,15,10,3,97,naples
2,2018-01-01,12,14,Cloudy,0.0,82,1014,54,25,10,3,97,naples
3,2018-01-01,18,12,Rain,0.7,63,1014,35,27,9,3,97,naples
4,2018-01-02,0,11,Rain,0.4,72,1014,35,26,9,3,100,naples


Weather data in Lecce

In [50]:
location = location_list[5]

dataset = RetrieveByAttribute(api_key, attributes, location, start_date, end_date, frequency).retrieve_hist_data()
dataset.to_csv('lecce.csv', encoding='utf-8', index=False)



Retrieving weather data for lecce


Retrieving data for lecce from: 2018-01-01 to: 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:00.971298
Retrieving data for lecce from: 2018-02-01 to: 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:01.986698
Retrieving data for lecce from: 2018-03-01 to: 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.939802
Retrieving data for lecce from: 2018-04-01 to: 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.958088
Retrieving data for lecce from: 2018-05-01 to: 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.884289
Retrieving data for lecce from: 2018-06-01 to: 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.810631
Retrieving data for lecce from: 2018-07-01 to: 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.863385
Retrieving data for lecce from: 2018-08-01 to: 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.094273
Retrieving data for lecce from: 2018-09-01 to: 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.133932
Retrieving data for lecce from: 2018-10-01 to: 2018-10-31
Time

In [51]:
lecceData = pd.read_csv('lecce.csv')
print(lecceData.shape)
lecceData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,11,[{'value': 'Clear'}],0.0,73,1021,7,14,10,4,97,lecce
1,2018-01-01,6,14,[{'value': 'Partly cloudy'}],0.0,87,1019,39,18,10,4,97,lecce
2,2018-01-01,12,16,[{'value': 'Cloudy'}],0.0,75,1017,83,15,10,4,97,lecce
3,2018-01-01,18,13,[{'value': 'Partly cloudy'}],0.0,79,1014,55,14,10,4,97,lecce
4,2018-01-02,0,13,[{'value': 'Clear'}],0.2,68,1013,0,33,10,4,100,lecce


In [52]:
#lecceData = pd.read_csv('lecce.csv')
tmp = []
conditions = ['Sunny','Clear','Cloudy','Rain','Snow']
for i, row in lecceData.iterrows():
  for j in conditions:
    if j.lower() in row['weatherDesc'].lower():
      lecceData.at[i, 'weatherDesc'] = j
      break
  else:
      tmp.append(i) if i not in tmp else None
lecceData = lecceData.drop(tmp)
lecceData = lecceData.drop_duplicates()
lecceData = lecceData.reset_index(drop=True)
lecceData.to_csv('lecce.csv', index=True)
lecceData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,11,Clear,0.0,73,1021,7,14,10,4,97,lecce
1,2018-01-01,6,14,Cloudy,0.0,87,1019,39,18,10,4,97,lecce
2,2018-01-01,12,16,Cloudy,0.0,75,1017,83,15,10,4,97,lecce
3,2018-01-01,18,13,Cloudy,0.0,79,1014,55,14,10,4,97,lecce
4,2018-01-02,0,13,Clear,0.2,68,1013,0,33,10,4,100,lecce


Weather data in Palermo

In [53]:
location = location_list[6]

dataset = RetrieveByAttribute(api_key, attributes, location, start_date, end_date, frequency).retrieve_hist_data()
dataset.to_csv('palermo.csv', encoding='utf-8', index=False)



Retrieving weather data for palermo


Retrieving data for palermo from: 2018-01-01 to: 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:00.958515
Retrieving data for palermo from: 2018-02-01 to: 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:01.886415
Retrieving data for palermo from: 2018-03-01 to: 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.850579
Retrieving data for palermo from: 2018-04-01 to: 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.753812
Retrieving data for palermo from: 2018-05-01 to: 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.810540
Retrieving data for palermo from: 2018-06-01 to: 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.874412
Retrieving data for palermo from: 2018-07-01 to: 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.903893
Retrieving data for palermo from: 2018-08-01 to: 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.965847
Retrieving data for palermo from: 2018-09-01 to: 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.002293
Retrieving data for palermo from: 2018-10-

In [54]:
palermoData = pd.read_csv('palermo.csv')
print(palermoData.shape)
palermoData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,11,[{'value': 'Partly cloudy'}],0.0,83,1023,25,9,10,4,97,palermo
1,2018-01-01,6,12,[{'value': 'Partly cloudy'}],0.0,69,1021,32,14,10,4,97,palermo
2,2018-01-01,12,16,[{'value': 'Sunny'}],0.0,70,1020,8,23,10,4,97,palermo
3,2018-01-01,18,11,[{'value': 'Clear'}],0.0,77,1021,0,19,10,4,97,palermo
4,2018-01-02,0,10,[{'value': 'Clear'}],0.0,80,1021,10,16,10,4,100,palermo


In [55]:
#palermoData = pd.read_csv('palermo.csv')
tmp = []
conditions = ['Sunny','Clear','Cloudy','Rain','Snow']
for i, row in palermoData.iterrows():
  for j in conditions:
    if j.lower() in row['weatherDesc'].lower():
      palermoData.at[i, 'weatherDesc'] = j
      break
  else:
      tmp.append(i) if i not in tmp else None
palermoData = palermoData.drop(tmp)
palermoData = palermoData.drop_duplicates()
palermoData = palermoData.reset_index(drop=True)
palermoData.to_csv('palermo.csv', index=True)

print(palermoData.shape)
palermoData.head()

,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,11,Cloudy,0.0,83,1023,25,9,10,4,97,palermo
1,2018-01-01,6,12,Cloudy,0.0,69,1021,32,14,10,4,97,palermo
2,2018-01-01,12,16,Sunny,0.0,70,1020,8,23,10,4,97,palermo
3,2018-01-01,18,11,Clear,0.0,77,1021,0,19,10,4,97,palermo
4,2018-01-02,0,10,Clear,0.0,80,1021,10,16,10,4,100,palermo


Merge data

Se hai un numero limitato di dati per ogni città o questi dati sono molto simili tra loro, potrebbe essere più efficace utilizzare un unico modello per tutte le città. In questo modo, il modello avrà a disposizione un numero maggiore di dati per "imparare" e potrebbe quindi ottenere risultati migliori.

In [153]:
# creazione della tabella finale utilizzando il metodo concat()
merged_table = pd.concat([milanData, turinData, florenceData, romeData, naplesData, lecceData, palermoData])
merged_table = merged_table.reset_index(drop=True)
merged_table.to_csv('merged_table.csv', index=False)

print(merged_table.shape)
merged_table.head()

(48052, 13)


,date,time,tempC,weatherDesc,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination,city
0,2018-01-01,0,5,Rain,0.8,86,1017,100,1,9,1,97,milan
1,2018-01-01,6,4,Rain,3.8,94,1013,100,3,7,1,97,milan
2,2018-01-01,12,5,Rain,2.5,90,1012,58,11,8,1,97,milan
3,2018-01-02,0,4,Cloudy,0.0,81,1010,50,5,10,3,100,milan
4,2018-01-02,6,4,Sunny,0.0,79,1015,0,8,10,3,100,milan


Se hai un gran numero di dati per ogni città e questi dati sono abbastanza diversi tra loro, potrebbe essere più efficace utilizzare un modello per ogni città. In questo modo, ogni modello potrà "specializzarsi" sui dati della città specifica, il che potrebbe portare a risultati migliori.

In [ ]:
# potrebbe essere utile per fare una pipeline più complessa, tipo fare fare la predizione su ogni città in modo separato

###Data Preprocessing

In [239]:
merged_table[['date',	'time','tempC',	'weatherDesc','precipMM', 'humidity','pressure',	'cloudcover',	'windspeedKmph','visibility',	'uvIndex','moon_illumination', 'city']] = merged_table[['date',	'time','tempC',	'weatherDesc','precipMM', 'humidity','pressure',	'cloudcover',	'windspeedKmph','visibility',	'uvIndex','moon_illumination', 'city']].replace(-0,0)

In [238]:
#DA NON ESEGUIRE

'''
Utilizzare lo scaler per standardizzare i dati della variabile indipendente sarebbe più appropriato, 
per modello di Random Forest che è un modello di classificazione basato sull'albero decisionale. 
Se invece stessi utilizzando un modello di regressione, allora potresti voler utilizzare il metodo factorize() 
per trasformare la variabile dipendente in una variabile numerica.
'''
# lettura di un file CSV in un DataFrame
df = pd.read_csv('merged_table.csv')

# creazione della variabile dipendente utilizzando il metodo factorize()
factor = pd.factorize(df['weatherDesc'])
values = factor[0]
definitions = factor[1]

uniq_values = np.unique(values)
print(uniq_values)
print(definitions)

[0 1 2 3 4]
Index(['Rain', 'Cloudy', 'Sunny', 'Clear', 'Snow'], dtype='object')


In [268]:
# lettura di un file CSV in un DataFrame
df = pd.read_csv('merged_table.csv')

#df = df.drop(['date','time'], axis=1) #La città potrebbe essere una feature più importante, poiché il meteo può variare significativamente da una città all'altra
df = df.drop(['date','time','city'], axis=1) # Tuttavia, se si utilizza un modello di Random Forest su un dataset di diverse città, il modello potrebbe essere in grado di apprendere autonomamente queste differenze e quindi la colonna 'city' potrebbe non essere necessaria.

'''
factor = pd.factorize(df['city'])
city_values = factor[0]
city_definitions = factor[1]

df['city'] = city_values
'''

y = df[['weatherDesc']]
x = df.drop(['weatherDesc'], axis=1)
columns = x.columns

# Standardize the data in X using a scaler
scaler = StandardScaler()
scaler.fit(x)
X = scaler.transform(x)
features = pd.DataFrame(X, columns = columns)

features


,tempC,precipMM,humidity,pressure,cloudcover,windspeedKmph,visibility,uvIndex,moon_illumination
0,-1.559415,0.233309,1.378782,0.145204,2.399513,-1.145327,-0.823538,-2.329272,1.624347
1,-1.685449,2.074483,1.895589,-0.440931,2.399513,-0.869912,-3.061050,-2.329272,1.624347
2,-1.559415,1.276641,1.637185,-0.587465,1.006490,0.231749,-1.942294,-2.329272,1.624347
3,-1.685449,-0.257671,1.055777,-0.880533,0.741153,-0.594497,0.295218,-1.150084,1.720088
4,-1.685449,-0.257671,0.926575,-0.147863,-0.917207,-0.181374,0.295218,-1.150084,1.720088
...,...,...,...,...,...,...,...,...,...
48047,-0.425107,-0.257671,0.797373,1.610543,-0.817705,-0.594497,0.295218,0.029104,0.028675
48048,-0.677176,-0.257671,0.409768,1.757076,-0.386532,-0.319081,0.295218,0.029104,0.156329
48049,-0.677176,-0.257671,-0.623847,1.903610,-0.419699,-0.456789,0.295218,0.029104,0.156329
48050,0.079030,-0.257671,-0.946851,2.050144,-0.685037,-0.594497,0.295218,0.029104,0.156329


In [283]:
#DA ESEGUIRE SU KUBEFLOW
dump(scaler, '../models/scaler.joblib')

['../models/scaler.joblib']

### Modelling

In [270]:
# Creating the Training and Test set from data
#X_train, X_test, y_train, y_test = train_test_split(features, y, test_size = 0.25, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size = 0.25, random_state = 21)


Tuttavia, alcune regole generali possono essere utili nella scelta dei parametri per un modello di Random Forest:

**bootstrap**: questo parametro specifica se i campioni per ogni albero della foresta devono essere estratti con o senza sostituzione. Di solito, impostare bootstrap = True è raccomandato, poiché questo può aiutare a ridurre la varianza del modello e aumentarne la robustezza.

**max_features**: questo parametro specifica il numero di features da considerare per ogni split durante l'addestramento di ogni albero della foresta. Un valore di 'sqrt' significa che il numero di features considerate per ogni split sarà la radice quadrata del numero totale di features nel dataset. Questo può essere un buon valore di default, poiché può aiutare a evitare il sovraccarico del modello.

**n_estimators**: questo parametro specifica il numero di alberi presenti nella foresta. Un valore più alto può portare a un modello più accurato, ma può anche aumentare i tempi di addestramento e di previsione. È importante trovare un buon compromesso tra accuratezza e tempi di esecuzione.

**criterion**: questo parametro specifica il criterio utilizzato per fare gli split durante l'addestramento di ogni albero della foresta. I due valori possibili sono 'entropy' e 'gini'. Entropy è basata sulla misura di impurità di un set di dati, mentre gini si basa sulla misura di purezza. In genere, non c'è un criterio migliore in assoluto per tutti i problemi di previsione meteorologica. Tuttavia, entropy può essere preferita in alcuni casi, poiché può fornire una maggiore flessibilità nella scelta dei parametri.

I criteri che si possono utilizzare con il Random Forest per la predizione del meteo sono:

*entropy*: il criterio di entropia, che si basa sulla misura di impurità di una particolare divisione di dati. Questo criterio cerca di minimizzare l'entropia del sotto-insieme di dati generato dalla divisione di un nodo.

*gini*: il criterio di impurità di Gini, che misura la probabilità che un elemento casuale appartenente a un insieme di dati scelto a caso sia classificato in modo errato. Il criterio di impurità di Gini cerca di minimizzare la probabilità di classificazione errata.

*mse*: il criterio di errore quadratico medio, che viene utilizzato per la regressione. In questo caso, il criterio cerca di minimizzare l'errore quadratico medio tra le previsioni del modello e i valori effettivi.

In [271]:
# Fitting Random Forest Classification to the Training set
model = RandomForestClassifier(n_estimators = 100 , bootstrap = True, max_features = 'sqrt', criterion = 'entropy')
model.fit(X_train, y_train)


<ipython-input-271-a9886d0ab87b>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', max_features='sqrt')

In [272]:
# Use the model to predict the classes of 'weatherDesc' on the test set
y_pred = model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.69


Per migliorare il modello di predizione del tempo tramite Random Forest, ci sono alcuni parametri che potresti modificare:

**n_estimators**: il numero di alberi nella foresta. Aumentare il numero di alberi può migliorare la precisione del modello, ma può anche aumentare il tempo di addestramento.

**criterion**: il criterio utilizzato per dividere i nodi durante il processo di costruzione degli alberi. Potresti provare a utilizzare altri criteri, come il coefficiente di Gini o il numero di osservazioni.

**max_depth**: la massima profondità degli alberi nella foresta. Una profondità più alta può portare a una maggiore precisione del modello, ma può anche aumentare il tempo di addestramento e il rischio di overfitting.

Inoltre, potresti considerare di aggiungere altre features al modello, come l'indice UV o l'illuminazione della luna, per vedere se queste possono aumentare la precisione del modello. Potresti anche considerare di utilizzare un modello diverso, come una reta neurale o una regressione logistica, per vedere se questi modelli danno risultati migliori.

**Rete neurale recurrente (RNN):** questo tipo di rete neurale viene utilizzato per il processamento dei dati sequenziali, come i dati meteorologici che vengono raccolti nel tempo. Una RNN consiste in una serie di neuroni che sono in grado di "ricordare" i dati delle precedenti iterazioni e di utilizzarli per influire sui risultati dell'iterazione corrente.

Re**gressione logistica multinomiale:** questo modello di regressione logistica viene utilizzato per prevedere la probabilità di appartenenza a più di due classi. Potrebbe essere utile per prevedere il meteo in più di due categorie, ad esempio se il tempo sarà nuvoloso, piovoso o soleggiato.



In [273]:
#DA ESEGUIRE SU KUBEFLOW
dump(model, '../models/model.joblib')

['../models/model.joblib']

### Inference

In [274]:
tempC = 18
precipMM = 0
humidity = 75
pressure = 1026
cloudcover = 45
windspeedKmph = 11
visibility = 10
uvIndex = 4
moon_illumination = 47

feat_cols = features.columns
row = [tempC, precipMM , humidity , pressure ,	cloudcover,	windspeedKmph , visibility, uvIndex, moon_illumination]

In [284]:
scaler = load('../models/scaler.joblib')
model = load('../models/model.joblib')

In [285]:
print(feat_cols)
print(conditions)

Index(['tempC', 'precipMM', 'humidity', 'pressure', 'cloudcover',
       'windspeedKmph', 'visibility', 'uvIndex', 'moon_illumination'],
      dtype='object')
['Sunny', 'Clear', 'Cloudy', 'Rain', 'Snow']


In [286]:
df = pd.DataFrame([row], columns = feat_cols)

X = scaler.transform(df)
features = pd.DataFrame(X, columns = feat_cols)

In [288]:
pred_condition = model.predict(features)
for condition in conditions:
  if pred_condition == condition:
    print(f'Il tempo previsto sarà: {pred_condition}')

Il tempo previsto sarà: ['Cloudy']


E' possibile utilizzare un modello di Random Forest per prevedere la città in cui si trova l'utente in base al meteo attuale utilizzando Streamlit. In particolare, potresti utilizzare le API di geolocalizzazione del browser o di un servizio di geolocalizzazione esterno per ottenere la posizione attuale dell'utente e utilizzare questi dati come input per il modello di Random Forest insieme ai dati meteorologici attuali. Una volta addestrato il modello, potresti utilizzare Streamlit per creare un'interfaccia utente che consenta all'utente di inserire i dati meteorologici attuali e ottenere una previsione della città in cui si trova.

Tieni presente che per ottenere buoni risultati con questo tipo di modello potrebbero essere necessari un dataset di addestramento di buone dimensioni con dati meteorologici accurati e una buona scelta delle features e dei parametri del modello. Inoltre, potrebbero essere necessari alcuni tentativi e aggiustamenti per ottenere un modello che funzioni correttamente.

In [ ]:
#PER STREAMLIT

'''Per utilizzare le API di geolocalizzazione del browser con Streamlit, 
puoi utilizzare il modulo streamlit.js e la funzione get_current_location().
'''
import streamlit as st

# Utilizza il modulo streamlit.js per ottenere la posizione attuale dell'utente
position = st.button("Get current location")
if position:
  location = st.js.get_current_location()
  latitude = location.latitude
  longitude = location.longitude
  st.write(f"La tua posizione attuale è: {latitude}, {longitude}")

'''Se vuoi utilizzare un servizio di geolocalizzazione esterno, puoi fare una richiesta HTTP a 
tale servizio utilizzando il modulo requests di Python. Ad esempio, puoi utilizzare l'API di geolocalizzazione di Google Maps come segue:'''

import requests

# Invia una richiesta all'API di Google Maps per ottenere la posizione attuale dell'utente
response = requests.get("https://www.googleapis.com/geolocation/v1/geolocate?key=YOUR_API_KEY")
location = response.json()
latitude = location['location']['lat']
longitude = location['location']['lng']

print(f"La tua posizione attuale è: {latitude}, {longitude}")

###OLD

LETTURA DEI CSV DA GOOGLE DRIVE


In [ ]:
url ='https://drive.google.com/file/d/1W6E6MBOOzMFiblLh6LrpQvVoQtsTyBrs/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path,delimiter=';', index_col=None, header=None,skiprows=1)
df.columns = ['date','city','tempC','precipMM', 'humidity', 'pressure','cloudcover', 'windspeedKmph', 'visibility', 'uvIndex','condition']

print('Shape of the dataset: ' + str(df.shape))
print(df.head())


In [ ]:
#Creating the dependent variable class
factor = pd.factorize(df['condition'])
dataset.condition = factor[0]
definitions = factor[1]
print(dataset.condition.head())
print(definitions)

In [ ]:
#Splitting the data into independent and dependent variables
df = df.drop(['date','time'], axis=1) #La città potrebbe essere una feature più importante, poiché il meteo può variare significativamente da una città all'altra
#df = df.drop(['date','time','city'], axis=1) # Tuttavia, se si utilizza un modello di Random Forest su un dataset di diverse città, il modello potrebbe essere in grado di apprendere autonomamente queste differenze e quindi la colonna 'city' potrebbe non essere necessaria.
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values
print('The independent features set: ')
print(X[:9,:])
print('The dependent variable: ')
print(y[:9])


In [ ]:
# Feature Scaling
print(df.head())
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

   tempC  precipMM  humidity  pressure  cloudcover  windspeedKmph  visibility  \
0      0       0.2        90      1016          33              3           9   
1      8       0.1        64      1015           0              4          10   
2      2       0.1        74      1012           3              7          10   
3      8       0.0        36      1019           6             14          10   
4      3       0.4        59      1018          23              0           9   

   uvIndex condition  
0        2     rainy  
1        2     sunny  
2        3     clear  
3        3     sunny  
4        1     rainy  


In [ ]:
# Predicting the Test set results
y_pred = model.predict(X_test)
#Reverse factorize (converting y_pred from 0s,1s and 2s to Iris-setosa, Iris-versicolor and Iris-virginica
reversefactor = dict(zip(range(6),definitions))
y_test = np.vectorize(reversefactor.get)(y_test)
y_pred = np.vectorize(reversefactor.get)(y_pred)
# Making the Confusion Matrix
print(pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted']))

Empty DataFrame
Columns: []
Index: []


In [ ]:
# Storing the trained model
print(list(zip(dataset.columns[0:4], model.feature_importances_)))
joblib.dump(model, 'randomforestmodel.pkl') 